<a href="https://colab.research.google.com/github/Aman-Agrawal01/RF/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTING NECESSARY LIBRARIES
Importing numpy, pandas, scikit learn for testing


In [1]:
import numpy as np
import pandas as pd
import random
import math
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

# LOADING IRIS DATA SET 
The dataset consists of 4 features of 3 classes of 50 instances each 



Link - https://archive.ics.uci.edu/ml/datasets/iris

In [2]:
iris = load_iris()
iris_features = pd.DataFrame(iris.data,columns=['sepal length','sepal width','petal length','petal width'])
iris_target = pd.DataFrame(iris.target,columns=['class'])
df = pd.concat([iris_features,iris_target], axis=1)
df = shuffle(df)
print(df.head())

     sepal length  sepal width  petal length  petal width  class
32            5.2          4.1           1.5          0.1      0
78            6.0          2.9           4.5          1.5      1
66            5.6          3.0           4.5          1.5      1
124           6.7          3.3           5.7          2.1      2
71            6.1          2.8           4.0          1.3      1


Splitting the data into test and training data

In [3]:
X_train = df[['sepal length','sepal width','petal length','petal width']][0:100].values
X_test = df[['sepal length','sepal width','petal length','petal width']][100:150].values
y_train = df['class'][0:100].values
y_test = df['class'][100:150].values

# CREATED NECESSARY FUNCTIONS

In [4]:
def random_forest_bootstrap(X, y, n_trees):
  forest = []
  indices = list(np.random.choice(range(len(X)), len(X), replace = True))
  for i in range(n_trees):
    X_bootstrap, y_bootstrap = X[indices], y[indices]
    tree = DecisionTreeClassifier()
    tree.fit(X_bootstrap, y_bootstrap)
    forest.append(tree)
  return forest

def random_forest_bagging(X, y, n_trees):
  forest = []
  indices = [random.sample(range(4), math.ceil(math.sqrt(4))) for i in range(n_trees)]
  for i in range(n_trees):
    X_bagging, y_bagging = X[:,indices[i]], y
    tree = DecisionTreeClassifier()
    tree.fit(X_bagging, y_bagging)
    forest.append(tree)
  return forest, indices

def predicting_random_forest_bootstrap(forest, X):
    prediction_list = list()
    for i in range(len(X)):
        ensemble_prediction = [tree.predict(X[i].reshape(1,-1)) for tree in forest]
        final_prediction = max(ensemble_prediction, key = ensemble_prediction.count)
        prediction_list.append(final_prediction)
    return np.array(prediction_list)

def predicting_random_forest_bagging(forest, X, indices):
    prediction_list = list()
    for i in range(len(X)):
      ensemble_prediction = []
      n = 0
      for tree in forest:
        ensemble_prediction.append(tree.predict(X[i,indices[n]].reshape(1,-1)))
        n = n + 1
      final_prediction = max(ensemble_prediction, key = ensemble_prediction.count)
      prediction_list.append(final_prediction)
    return np.array(prediction_list)

# TESTING

In [5]:
n_trees = 10

In [6]:
forest_bootstrap = random_forest_bootstrap(X_train, y_train, n_trees = n_trees)

prediction = predicting_random_forest_bootstrap(forest_bootstrap,X_test).reshape(1,-1)

accuracy = np.sum(sum(prediction == y_test))/len(y_test)
print(accuracy)

0.92


In [7]:
forest_bagging, indices = random_forest_bagging(X_train, y_train, n_trees = n_trees)

prediction = predicting_random_forest_bagging(forest_bagging,X_test,indices).reshape(1,-1)

accuracy = np.sum(sum(prediction == y_test))/len(y_test)
print(accuracy)

0.92


In [8]:
rf_bootstrap = RandomForestClassifier(n_estimators = n_trees, bootstrap = True)
rf_bootstrap.fit(X_train,y_train)

print(rf_bootstrap.score(X_test,y_test))

0.92


In [9]:
rf_bagging = BaggingClassifier(base_estimator = RandomForestClassifier(), n_estimators = n_trees)
rf_bagging.fit(X_train,y_train)

print(rf_bagging.score(X_test,y_test))

0.92
